In [36]:
import urllib.request

url = r'https://mp.weixin.qq.com/s/0P2XRFCN7d7MTMQQtScr7A'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53'
}
request = urllib.request.Request(url=url, headers=headers)
response = urllib.request.urlopen(request)
page = response.read().decode('utf-8')
with open('soft.html', 'w', encoding='utf-8') as f:
    f.write(page)
print(len(page))

In [2]:
# 读取下载的html
with open('soft.html', 'r', encoding='utf-8') as f:
    f.seek(0)
    html_doc = ''.join(f.read())
# print(html_doc)
# print(type(html_doc))
print(len(html_doc))

624411


In [3]:
# 导入所需库
import re

# 匹配a标签
all_A = re.findall(r'<a\b[^>]+\bhref="([^"]*)"[^>]*>([\s\S]*?)</a>', html_doc)
# print(all_A)
print(len(all_A))

698


In [5]:
# 测试
name = re.findall(r"<span.*>(.*?)</span>", str(all_A[350]))
print(name)
print(all_A[350])
print(all_A[1])

['V-Ray5.2']
('http://mp.weixin.qq.com/s?__biz=MzA4MjU4MTg2Ng==&amp;mid=2247498667&amp;idx=4&amp;sn=d8080f496ae298283ca83f3b7e472dc0&amp;chksm=9f813baea8f6b2b86de1ceef931b3ac2e721283b3f9048422fff86bb05da59a880a4a87bf6f5&amp;scene=21#wechat_redirect', '<span style="font-size: 14px;">V-Ray5.2</span>')
('https://mp.weixin.qq.com/mp/homepage?__biz=MzA4MjU4MTg2Ng==&amp;hid=18&amp;sn=2dccf8223c0bac443c2b8f260731278a', '「软件管家」')


In [4]:
# 处理匹配后的a标签
nameHrefDict = {}
for key, value in enumerate(all_A):
    if value[0].startswith('http://mp.weixin.qq.com/s?__biz=MzA4MjU4MTg2Ng==&amp'):
        name = ''.join(re.findall(r"<span.*?>(.*?)</span>", value[1]))
        if name == '':
            name = value[1]
        # print(key, name, value[0])
        nameHrefDict[name] = value[0]
print(len(nameHrefDict))  # 660
# nameHrefDict

678


In [42]:
# 存入数据库
import pymysql

config = {
    'host'    : "localhost",
    'port'    : 3306,
    'user'    : "root",
    'passwd'  : "031214",
    'charset' : 'utf8',
    'database': 'mydata',
}
connect = pymysql.connect(**config)
# 获取游标对象
cursor = connect.cursor()
# 提交MySQL命令
for soft_name in nameHrefDict:
    soft_url = nameHrefDict[soft_name]
    cursor.execute("insert into mydata.soft(name, url) values(%s, %s)", (soft_name, soft_url))
# 提交事务
connect.commit()
# 关闭游标
cursor.close()
# 关闭连接
connect.close()